In [188]:
import numpy as np
import pandas as pd
import datetime
from datetime import date, datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings

# import workalendar
# from workalendar.america import Brazil

warnings.filterwarnings('ignore')
np.random.seed(4590)

skf= StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)

# by feature select 
# FILTER_FEATURE = ['new_hist_month_diff_min_-_hist_month_diff_min', 'new_hist_merchant_category_id_mean_mean_add_hist_merchant_category_id_mean_mean', 'new_hist_month_lag_mean_mean_add_hist_month_lag_mean_mean', 'hist_year_nunique', 'new_hist_month_diff_max_/_hist_month_diff_max', 'new_hist_merchant_id_mean_mean_add_hist_merchant_id_mean_mean', 'new_hist_month_diff_min_/_hist_month_diff_min', 'new_hist_year_mean_mean_add_hist_year_mean_mean', 'new_hist_dayofweek_mean_mean_/_hist_dayofweek_mean_mean', 'new_hist_weekofyear_mean_mean_add_hist_weekofyear_mean_mean', 'new_hist_dayofweek_nunique_/_hist_dayofweek_nunique', 'new_hist_month_diff_max', 'new_hist_year_mean_mean_/_hist_year_mean_mean', 'new_hist_merchant_category_id_mean_mean_/_hist_merchant_category_id_mean_mean', 'new_hist_merchant_id_mean_mean_/_hist_merchant_id_mean_mean', 'new_hist_year_nunique_/_hist_year_nunique', 'new_hist_month_diff_min', 'new_hist_merchant_category_id_mean_mean', 'hist_month_diff_mean_hist_month_diff_min_ctr', 'new_hist_year_nunique_-_hist_year_nunique', 'dayofweek', 'new_hist_month_mean_mean_/_hist_month_mean_mean', 'new_hist_month_diff_var', 'new_hist_installments_min_-_hist_installments_min', 'new_hist_subsector_id_mean_mean', 'new_hist_dayofweek_nunique', 'new_hist_weekend_mean', 'new_hist_category_2_mean_mean_/_hist_category_2_mean_mean', 'new_hist_merchant_category_id_nunique_-_hist_merchant_category_id_nunique', 'new_hist_month_diff_mean_add_hist_month_diff_mean', 'new_hist_authorized_flag_mean', 'new_hist_merchant_category_id_mean_mean_-_hist_merchant_category_id_mean_mean', 'new_hist_year_mean_mean_-_hist_year_mean_mean', 'new_hist_category_2_mean_mean', 'new_hist_card_id_mean_mean_add_hist_card_id_mean_mean', 'new_hist_month_diff_max_-_hist_month_diff_max', 'new_hist_weekend_sum']

In [189]:
sDir = '/home/pabhijit/data/sample/'

In [190]:
df_train = pd.read_csv(sDir + 'train.csv')
df_test = pd.read_csv(sDir + 'test.csv')
df_hist_trans = pd.read_csv(sDir + 'historical_transactions.csv',parse_dates=['purchase_date'])
df_new_merchant_trans = pd.read_csv(sDir + 'new_merchant_transactions.csv',parse_dates=['purchase_date'])

In [191]:
max_date=df_new_merchant_trans['purchase_date'].max()

In [192]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [193]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

### Step-1 (Parse "purchase_date")

In [194]:
for df in [df_hist_trans,df_new_merchant_trans]:
#     df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['day'] = df['purchase_date'].dt.day
    
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    df['month_diff'] = ((max_date - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']                                                                           

### Step-2 (Aggregate Features)

In [195]:
def  get_agg_fea(count_df,prefix):
    aggs = {}
    for col in ['month','day','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum','max','min','mean','var']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var']
    # aggs['month_diff'] = ['mean']
    aggs['month_diff'] = ['max','min','mean','var']
    aggs['authorized_flag'] = ['sum', 'mean']
    aggs['weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']

    for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',\
                'category_1','category_2','category_3','month_lag','card_id']:
        count_df[col+'_mean'] = count_df.groupby([col])['purchase_amount'].transform('mean')
        aggs[col+'_mean'] = ['mean']   

    new_columns = get_new_columns(prefix,aggs)
    print('aggs : ', aggs)
    print('new_columns : ', new_columns)
    count_df_gp = count_df.groupby('card_id').agg(aggs)
    count_df_gp.columns = new_columns
    count_df_gp.reset_index(drop=False,inplace=True)
    count_df_gp['%s_purchase_date_diff'%prefix] = (count_df_gp['%s_purchase_date_max'%prefix] - count_df_gp['%s_purchase_date_min'%prefix]).dt.days
    count_df_gp['%s_purchase_date_average'%prefix] = count_df_gp['%s_purchase_date_diff'%prefix]/count_df_gp['%s_card_id_size'%prefix]
    count_df_gp['%s_purchase_date_uptonow'%prefix] = (max_date - count_df_gp['%s_purchase_date_max'%prefix]).dt.days
    count_df_gp['%s_purchase_date_uptomin'%prefix] = (max_date - count_df_gp['%s_purchase_date_min'%prefix]).dt.days
    
    return count_df_gp

In [196]:
print('df_train.shape : ', df_train.shape)
print('df_test.shape : ', df_test.shape)
print('df_train.columns : ', df_train.columns)
print('df_test.columns : ', df_test.columns)

df_train.shape :  (10, 6)
df_test.shape :  (10, 5)
df_train.columns :  Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target'],
      dtype='object')
df_test.columns :  Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3'], dtype='object')


### Step-2.1 : Merge Aggregated Historical Transactions to Train and Test

In [197]:
hist_count_df=get_agg_fea(df_hist_trans,'hist')
df_train = df_train.merge(hist_count_df,on='card_id',how='left')
df_test = df_test.merge(hist_count_df,on='card_id',how='left')
#del hist_count_df
gc.collect()

aggs :  {'month': ['nunique'], 'day': ['nunique'], 'hour': ['nunique'], 'weekofyear': ['nunique'], 'dayofweek': ['nunique'], 'year': ['nunique'], 'subsector_id': ['nunique'], 'merchant_id': ['nunique'], 'merchant_category_id': ['nunique'], 'purchase_amount': ['sum', 'max', 'min', 'mean', 'var'], 'installments': ['sum', 'max', 'min', 'mean', 'var'], 'purchase_date': ['max', 'min'], 'month_lag': ['max', 'min', 'mean', 'var'], 'month_diff': ['max', 'min', 'mean', 'var'], 'authorized_flag': ['sum', 'mean'], 'weekend': ['sum', 'mean'], 'category_1': ['sum', 'mean'], 'card_id': ['size'], 'month_mean': ['mean'], 'hour_mean': ['mean'], 'weekofyear_mean': ['mean'], 'dayofweek_mean': ['mean'], 'year_mean': ['mean'], 'subsector_id_mean': ['mean'], 'merchant_id_mean': ['mean'], 'merchant_category_id_mean': ['mean'], 'category_1_mean': ['mean'], 'category_2_mean': ['mean'], 'category_3_mean': ['mean'], 'month_lag_mean': ['mean'], 'card_id_mean': ['mean']}
new_columns :  ['hist_month_nunique', 'hist

263

In [199]:
hist_count_df.head()

,card_id,hist_month_nunique,hist_day_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_year_nunique,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,...,hist_merchant_category_id_mean_mean,hist_category_1_mean_mean,hist_category_2_mean_mean,hist_category_3_mean_mean,hist_month_lag_mean_mean,hist_card_id_mean_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_purchase_date_uptomin
0,C_ID_0894217f2f,7,17,13,16,6,1,8,22,14,...,-0.527610,-0.553394,-0.616044,-0.170294,-0.513861,-0.417462,199,6.030303,273,473
1,C_ID_0ab67a22ab,9,24,17,24,7,1,12,24,16,...,-0.535758,-0.434259,-0.615316,-0.385718,-0.524808,-0.599025,268,3.941176,121,389
2,C_ID_130fd0cbdd,5,27,18,20,7,2,12,27,16,...,-0.539192,-0.555198,-0.393945,-0.533565,-0.581034,-0.629955,401,5.141026,69,470
3,C_ID_186d6a6901,6,25,16,20,7,2,13,50,25,...,-0.560590,-0.504824,-0.436702,-0.499968,-0.476746,-0.642745,154,2.000000,59,214
4,C_ID_2b5e3df5c2,12,27,21,34,7,2,15,47,31,...,-0.420521,-0.565118,-0.396227,-0.519220,-0.541705,0.228540,419,3.809091,61,480


In [132]:
print('df_train.shape : ', df_train.shape)
print('df_test.shape : ', df_test.shape)
print('df_train.columns : ', df_train.columns)
print('df_test.columns : ', df_test.columns)

df_train.shape :  (10, 59)
df_test.shape :  (10, 58)
df_train.columns :  Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target', 'hist_month_nunique', 'hist_day_nunique', 'hist_hour_nunique',
       'hist_weekofyear_nunique', 'hist_dayofweek_nunique',
       'hist_year_nunique', 'hist_subsector_id_nunique',
       'hist_merchant_id_nunique', 'hist_merchant_category_id_nunique',
       'hist_purchase_amount_sum', 'hist_purchase_amount_max',
       'hist_purchase_amount_min', 'hist_purchase_amount_mean',
       'hist_purchase_amount_var', 'hist_installments_sum',
       'hist_installments_max', 'hist_installments_min',
       'hist_installments_mean', 'hist_installments_var',
       'hist_purchase_date_max', 'hist_purchase_date_min',
       'hist_month_lag_max', 'hist_month_lag_min', 'hist_month_lag_mean',
       'hist_month_lag_var', 'hist_month_diff_max', 'hist_month_diff_min',
       'hist_month_diff_mean', 'hist_month_diff_var',
       'hist_au

### Step-2.2 : Merge Aggregated "New Merchant Transactions" to Train and Test

In [133]:
new_count_df = get_agg_fea(df_new_merchant_trans,'new_hist')
df_train = df_train.merge(new_count_df,on='card_id',how='left')
df_test = df_test.merge(new_count_df,on='card_id',how='left')
del new_count_df
gc.collect()

31

In [134]:
print('df_train.shape : ', df_train.shape)
print('df_test.shape : ', df_test.shape)
print('df_train.columns : ', df_train.columns)
print('df_test.columns : ', df_test.columns)

df_train.shape :  (10, 112)
df_test.shape :  (10, 111)
df_train.columns :  Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target', 'hist_month_nunique', 'hist_day_nunique', 'hist_hour_nunique',
       'hist_weekofyear_nunique',
       ...
       'new_hist_merchant_category_id_mean_mean',
       'new_hist_category_1_mean_mean', 'new_hist_category_2_mean_mean',
       'new_hist_category_3_mean_mean', 'new_hist_month_lag_mean_mean',
       'new_hist_card_id_mean_mean', 'new_hist_purchase_date_diff',
       'new_hist_purchase_date_average', 'new_hist_purchase_date_uptonow',
       'new_hist_purchase_date_uptomin'],
      dtype='object', length=112)
df_test.columns :  Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'hist_month_nunique', 'hist_day_nunique', 'hist_hour_nunique',
       'hist_weekofyear_nunique', 'hist_dayofweek_nunique',
       ...
       'new_hist_merchant_category_id_mean_mean',
       'new_hist_c

### Step-3 (New Features)

In [135]:
for df in [df_train,df_test]:
    
    
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['first_active_monthtonoew'] = ( max_date-pd.to_datetime(df['first_active_month'])).dt.days

    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (max_date- df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['hist_sleep_day'] = df['first_active_monthtonoew'] - df['hist_day_nunique']
    df['new_sleep_day'] = df['first_active_monthtonoew'] - df['new_hist_day_nunique']
    df['total_sleep_day'] = df['hist_sleep_day'] + df['new_sleep_day'] 
    df['diff_sleep_day'] = df['hist_sleep_day'] - df['new_sleep_day']
   
    
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']

    
    df['purchase_amount_diff'] = df['new_hist_purchase_amount_sum']-df['hist_purchase_amount_sum']
    df['purchase_amount_rate'] = df['purchase_amount_diff']/df['hist_purchase_amount_sum']

  

In [136]:
print('df_train.shape : ', df_train.shape)
print('df_test.shape : ', df_test.shape)
print('df_train.columns : ', df_train.columns)
print('df_test.columns : ', df_test.columns)

df_train.shape :  (10, 126)
df_test.shape :  (10, 125)
df_train.columns :  Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target', 'hist_month_nunique', 'hist_day_nunique', 'hist_hour_nunique',
       'hist_weekofyear_nunique',
       ...
       'elapsed_time', 'hist_first_buy', 'new_hist_first_buy',
       'hist_sleep_day', 'new_sleep_day', 'total_sleep_day', 'diff_sleep_day',
       'purchase_amount_total', 'purchase_amount_diff',
       'purchase_amount_rate'],
      dtype='object', length=126)
df_test.columns :  Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'hist_month_nunique', 'hist_day_nunique', 'hist_hour_nunique',
       'hist_weekofyear_nunique', 'hist_dayofweek_nunique',
       ...
       'elapsed_time', 'hist_first_buy', 'new_hist_first_buy',
       'hist_sleep_day', 'new_sleep_day', 'total_sleep_day', 'diff_sleep_day',
       'purchase_amount_total', 'purchase_amount_diff',
       'purchase_amo

### Step-4 (New Features "new_hist")

In [137]:
new_hist_feature=[ i for i in df_train  if 'new_hist' in i]

for fea in new_hist_feature:
    if fea in ['new_hist_purchase_date_max','new_hist_purchase_date_min']:
        continue
    print(fea)
    new_fea=fea
    hist_fea=fea.replace("new_hist",'hist')
    df_train[new_fea+"_add_"+hist_fea]=df_train.apply(lambda x: x[new_fea]+x[hist_fea],axis=1)
    df_test[new_fea+"_add_"+hist_fea]=df_test.apply(lambda x: x[new_fea]+x[hist_fea],axis=1)
    df_train[new_fea+"_-_"+hist_fea]=df_train.apply(lambda x: x[new_fea]-x[hist_fea],axis=1)
    df_test[new_fea+"_-_"+hist_fea]=df_test.apply(lambda x: x[new_fea]-x[hist_fea],axis=1)
    df_train[new_fea+"_/_"+hist_fea]=df_train.apply(lambda x: x[new_fea]/(abs(x[hist_fea])+0.01),axis=1)
    df_test[new_fea+"_/_"+hist_fea]=df_test.apply(lambda x: x[new_fea]/(abs(x[hist_fea])+0.01),axis=1)

new_hist_month_nunique
new_hist_day_nunique
new_hist_hour_nunique
new_hist_weekofyear_nunique
new_hist_dayofweek_nunique
new_hist_year_nunique
new_hist_subsector_id_nunique
new_hist_merchant_id_nunique
new_hist_merchant_category_id_nunique
new_hist_purchase_amount_sum
new_hist_purchase_amount_max
new_hist_purchase_amount_min
new_hist_purchase_amount_mean
new_hist_purchase_amount_var
new_hist_installments_sum
new_hist_installments_max
new_hist_installments_min
new_hist_installments_mean
new_hist_installments_var
new_hist_month_lag_max
new_hist_month_lag_min
new_hist_month_lag_mean
new_hist_month_lag_var
new_hist_month_diff_max
new_hist_month_diff_min
new_hist_month_diff_mean
new_hist_month_diff_var
new_hist_authorized_flag_sum
new_hist_authorized_flag_mean
new_hist_weekend_sum
new_hist_weekend_mean
new_hist_category_1_sum
new_hist_category_1_mean
new_hist_card_id_size
new_hist_month_mean_mean
new_hist_hour_mean_mean
new_hist_weekofyear_mean_mean
new_hist_dayofweek_mean_mean
new_hist_yea

In [138]:
print('df_train.shape : ', df_train.shape)
print('df_test.shape : ', df_test.shape)
print('df_train.columns : ', df_train.columns)
print('df_test.columns : ', df_test.columns)

df_train.shape :  (10, 282)
df_test.shape :  (10, 281)
df_train.columns :  Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target', 'hist_month_nunique', 'hist_day_nunique', 'hist_hour_nunique',
       'hist_weekofyear_nunique',
       ...
       'new_hist_purchase_date_average_/_hist_purchase_date_average',
       'new_hist_purchase_date_uptonow_add_hist_purchase_date_uptonow',
       'new_hist_purchase_date_uptonow_-_hist_purchase_date_uptonow',
       'new_hist_purchase_date_uptonow_/_hist_purchase_date_uptonow',
       'new_hist_purchase_date_uptomin_add_hist_purchase_date_uptomin',
       'new_hist_purchase_date_uptomin_-_hist_purchase_date_uptomin',
       'new_hist_purchase_date_uptomin_/_hist_purchase_date_uptomin',
       'new_hist_first_buy_add_hist_first_buy',
       'new_hist_first_buy_-_hist_first_buy',
       'new_hist_first_buy_/_hist_first_buy'],
      dtype='object', length=282)
df_test.columns :  Index(['first_active_month', 'car

### Step-5 (New Features "counters")

In [139]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    10
Name: outliers, dtype: int64

In [140]:
def get_data_ctr_fea(tj_data,self_data,items=['feature_1','feature_2','feature_3','new_hist_month_diff_mean',
                                          'hist_month_diff_mean', 'hist_month_diff_max','hist_month_diff_min' ]):

    items = items
    
    tj_drop_columns=[i for i in  tj_data.columns  if "_ctr" in i]
    if len(tj_drop_columns)>0:
        tj_data=tj_data.drop(columns=tj_drop_columns)
    
    self_drop_columns=[i for i in  self_data  if "_ctr" in i]
    if len(self_drop_columns)>0:
        print(self_drop_columns)
        self_data.drop(columns=self_drop_columns,inplace=True)
        
    
    for item in items:
        if type(item)==list:
            pr_name="_".join(item)
            merge_columns=item+[pr_name+'_ctr']
        else:
            pr_name=item
            merge_columns=[item,pr_name+'_ctr']
        temp = tj_data.groupby(item, as_index = False)['outliers'].agg({pr_name+'_click':'sum',pr_name+'_count':'count'})
        temp[pr_name+'_ctr'] =1000* (temp[pr_name+'_click']+0.01)/(temp[pr_name+'_count']+0.01)
        
        self_data = pd.merge(self_data, temp[merge_columns], on=item, how='left')
        
    for i in range(len(items)):
        for j in range(i+1, len(items)):
            item_g = [items[i], items[j]]
            merge_columns=item_g+['_'.join(item_g)+'_ctr']
            temp = tj_data.groupby(item_g, as_index=False)['outliers'].agg({'_'.join(item_g)+'_click': 'sum','_'.join(item_g)+'count':'count'})
            temp['_'.join(item_g)+'_ctr'] =1000* (temp['_'.join(item_g)+'_click']+0.01)/(temp['_'.join(item_g)+'count']+0.01)
            self_data = pd.merge(self_data, temp[merge_columns], on=item_g, how='left')
            
    return self_data
            
df_test=get_data_ctr_fea(df_train,df_test)  
print("df_test ctr caluate finished")

from sklearn.model_selection import  StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=4590, shuffle=True)

df_train['index'] = [i for i in range(len(df_train))]

for k, (df_train_in, df_test_in) in enumerate(skf.split(df_train, df_train.outliers.values)):
    df_train_df=df_train.iloc[df_train_in]
    val_df=df_train.iloc[df_test_in]
    val_df=get_data_ctr_fea(df_train_df,val_df)
    if k==0:
        new_df_train_df=val_df
    else:
        new_df_train_df=pd.concat([new_df_train_df,val_df])

df_train=new_df_train_df
df_train.sort_values(by='index',inplace=True)
df_train.drop(columns = ['index'],inplace=True)

print("df_train ctr caluate finished")


df_test ctr caluate finished
df_train ctr caluate finished


In [141]:
print('df_train.shape : ', df_train.shape)
print('df_test.shape : ', df_test.shape)
print('df_train.columns : ', df_train.columns)
print('df_test.columns : ', df_test.columns)

df_train.shape :  (10, 311)
df_test.shape :  (10, 309)
df_train.columns :  Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target', 'hist_month_nunique', 'hist_day_nunique', 'hist_hour_nunique',
       'hist_weekofyear_nunique',
       ...
       'feature_3_new_hist_month_diff_mean_ctr',
       'feature_3_hist_month_diff_mean_ctr',
       'feature_3_hist_month_diff_max_ctr',
       'feature_3_hist_month_diff_min_ctr',
       'new_hist_month_diff_mean_hist_month_diff_mean_ctr',
       'new_hist_month_diff_mean_hist_month_diff_max_ctr',
       'new_hist_month_diff_mean_hist_month_diff_min_ctr',
       'hist_month_diff_mean_hist_month_diff_max_ctr',
       'hist_month_diff_mean_hist_month_diff_min_ctr',
       'hist_month_diff_max_hist_month_diff_min_ctr'],
      dtype='object', length=311)
df_test.columns :  Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'hist_month_nunique', 'hist_day_nunique', 'hist_hour_nuni

In [142]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_month_nunique,hist_day_nunique,hist_hour_nunique,hist_weekofyear_nunique,...,feature_3_new_hist_month_diff_mean_ctr,feature_3_hist_month_diff_mean_ctr,feature_3_hist_month_diff_max_ctr,feature_3_hist_month_diff_min_ctr,new_hist_month_diff_mean_hist_month_diff_mean_ctr,new_hist_month_diff_mean_hist_month_diff_max_ctr,new_hist_month_diff_mean_hist_month_diff_min_ctr,hist_month_diff_mean_hist_month_diff_max_ctr,hist_month_diff_mean_hist_month_diff_min_ctr,hist_month_diff_max_hist_month_diff_min_ctr
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,9,31,23,35,...,NaN,NaN,9.900990,9.900990,NaN,NaN,NaN,NaN,NaN,9.90099
0,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,12,31,24,50,...,NaN,NaN,NaN,4.975124,NaN,NaN,NaN,NaN,NaN,NaN
0,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,10,19,14,22,...,3.322259,NaN,9.900990,4.975124,NaN,4.975124,9.900990,NaN,NaN,NaN
1,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,6,25,16,20,...,3.322259,NaN,4.975124,4.975124,NaN,9.900990,4.975124,NaN,NaN,9.90099
0,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,4,30,22,17,...,NaN,NaN,9.900990,4.975124,NaN,NaN,NaN,NaN,NaN,9.90099


In [143]:
# Dealing with the one nan in df_test.first_active_month a bit arbitrarily for now
#df_test.loc[df_test['first_active_month'].isna(),'first_active_month'] = df_test.iloc[11577]['first_active_month']

In [144]:
bed_fea = ['new_hist_month_diff_min_-_hist_month_diff_min', 'new_hist_year_mean_mean_/_hist_year_mean_mean', 'new_hist_year_nunique_/_hist_year_nunique', 'new_hist_year_mean_mean_add_hist_year_mean_mean', 'new_hist_merchant_category_id_mean_mean_-_hist_merchant_category_id_mean_mean', 'new_hist_month_lag_mean_mean_-_hist_month_lag_mean_mean', 'new_hist_month_diff_max_add_hist_month_diff_max', 'new_hist_month_diff_min', 'new_hist_weekend_sum_/_hist_weekend_sum', 'new_hist_year_mean_mean', 'new_hist_merchant_id_mean_mean_add_hist_merchant_id_mean_mean', 'new_hist_purchase_amount_sum_-_hist_purchase_amount_sum', 'new_hist_month_diff_var', 'new_hist_month_diff_max_/_hist_month_diff_max', 'new_hist_weekofyear_nunique', 'new_hist_month_diff_max_-_hist_month_diff_max', 'new_hist_weekend_mean_add_hist_weekend_mean', 'new_hist_month_mean_mean_-_hist_month_mean_mean', 'new_hist_month_diff_mean', 'hist_year_nunique', 'new_hist_weekend_mean', 'new_hist_year_mean_mean_-_hist_year_mean_mean', 'hist_month_diff_mean_hist_month_diff_min_ctr', 'hist_dayofweek_nunique', 'new_hist_merchant_category_id_mean_mean', 'new_hist_weekend_sum', 'new_hist_installments_min_-_hist_installments_min', 'new_hist_authorized_flag_mean', 'new_hist_weekend_mean_/_hist_weekend_mean', 'hist_month_diff_min', 'new_hist_card_id_mean_mean_add_hist_card_id_mean_mean', 'new_hist_month_mean_mean_/_hist_month_mean_mean', 'new_hist_category_2_mean_mean', 'new_hist_month_diff_min_/_hist_month_diff_min', 'new_hist_merchant_category_id_mean_mean_add_hist_merchant_category_id_mean_mean', 'new_hist_dayofweek_nunique_/_hist_dayofweek_nunique', 'new_hist_month_mean_mean', 'new_hist_day_nunique_add_hist_day_nunique', 'new_hist_subsector_id_nunique_-_hist_subsector_id_nunique', 'new_hist_purchase_amount_max_add_hist_purchase_amount_max', 'new_hist_month_diff_mean_add_hist_month_diff_mean', 'new_hist_subsector_id_mean_mean', 'new_hist_year_nunique_-_hist_year_nunique', 'new_hist_dayofweek_nunique_-_hist_dayofweek_nunique', 'new_hist_merchant_id_mean_mean_/_hist_merchant_id_mean_mean']

In [145]:
# bed_fea =['new_hist_month_diff_min', 'new_hist_authorized_flag_mean', 'new_hist_year_mean_mean', 'new_hist_year_nunique_/_hist_year_nunique', 'new_hist_year_nunique_-_hist_year_nunique']
del_col =  ['card_id', 'first_active_month','target','outliers']+bed_fea

features = [c for c in df_train.columns if c not in del_col ]

target = df_train['target']

train_y = target 
# df_train.drop(columns  = ["card_id","target"],inplace=True)
# df_test.drop(columns = ["card_id"],inplace=True)

In [151]:
print(len(del_col))
print(len(features))
del_col

49
262


['card_id',
 'first_active_month',
 'target',
 'outliers',
 'new_hist_month_diff_min_-_hist_month_diff_min',
 'new_hist_year_mean_mean_/_hist_year_mean_mean',
 'new_hist_year_nunique_/_hist_year_nunique',
 'new_hist_year_mean_mean_add_hist_year_mean_mean',
 'new_hist_merchant_category_id_mean_mean_-_hist_merchant_category_id_mean_mean',
 'new_hist_month_lag_mean_mean_-_hist_month_lag_mean_mean',
 'new_hist_month_diff_max_add_hist_month_diff_max',
 'new_hist_month_diff_min',
 'new_hist_weekend_sum_/_hist_weekend_sum',
 'new_hist_year_mean_mean',
 'new_hist_merchant_id_mean_mean_add_hist_merchant_id_mean_mean',
 'new_hist_purchase_amount_sum_-_hist_purchase_amount_sum',
 'new_hist_month_diff_var',
 'new_hist_month_diff_max_/_hist_month_diff_max',
 'new_hist_weekofyear_nunique',
 'new_hist_month_diff_max_-_hist_month_diff_max',
 'new_hist_weekend_mean_add_hist_weekend_mean',
 'new_hist_month_mean_mean_-_hist_month_mean_mean',
 'new_hist_month_diff_mean',
 'hist_year_nunique',
 'new_hist_w

In [147]:
features

['feature_1',
 'feature_2',
 'feature_3',
 'hist_month_nunique',
 'hist_day_nunique',
 'hist_hour_nunique',
 'hist_weekofyear_nunique',
 'hist_subsector_id_nunique',
 'hist_merchant_id_nunique',
 'hist_merchant_category_id_nunique',
 'hist_purchase_amount_sum',
 'hist_purchase_amount_max',
 'hist_purchase_amount_min',
 'hist_purchase_amount_mean',
 'hist_purchase_amount_var',
 'hist_installments_sum',
 'hist_installments_max',
 'hist_installments_min',
 'hist_installments_mean',
 'hist_installments_var',
 'hist_purchase_date_max',
 'hist_purchase_date_min',
 'hist_month_lag_max',
 'hist_month_lag_min',
 'hist_month_lag_mean',
 'hist_month_lag_var',
 'hist_month_diff_max',
 'hist_month_diff_mean',
 'hist_month_diff_var',
 'hist_authorized_flag_sum',
 'hist_authorized_flag_mean',
 'hist_weekend_sum',
 'hist_weekend_mean',
 'hist_category_1_sum',
 'hist_category_1_mean',
 'hist_card_id_size',
 'hist_month_mean_mean',
 'hist_hour_mean_mean',
 'hist_weekofyear_mean_mean',
 'hist_dayofweek_m

In [148]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         'lambda_l2':0.1,
         "verbosity": -1,
         "n_jobs":10,
         "random_state": 4590}

In [157]:
gp = df_hist_trans.groupby('card_id')
for cid,cid_df in  gp:
   print(cid, cid_df)

C_ID_0894217f2f      authorized_flag          card_id  city_id  category_1  installments  \
863                1  C_ID_0894217f2f       69           0             3   
864                1  C_ID_0894217f2f       69           0             3   
865                1  C_ID_0894217f2f       69           0             1   
866                1  C_ID_0894217f2f       69           0             3   
867                1  C_ID_0894217f2f       69           0             6   
868                1  C_ID_0894217f2f       69           0             1   
869                1  C_ID_0894217f2f       69           0             3   
870                1  C_ID_0894217f2f       69           0             3   
871                0  C_ID_0894217f2f       69           0             1   
872                1  C_ID_0894217f2f       69           0             1   
873                1  C_ID_0894217f2f       69           0             1   
874                1  C_ID_0894217f2f       69           0             3

### Step-6 (New Text Features)

In [152]:
df_hist_ca=['authorized_flag','category_1','installments','category_3','merchant_category_id','merchant_id','category_2','state_id',\
          'subsector_id','purchase_amount' ]

hist_text_feature={}
gp=df_hist_trans.groupby('card_id')
for fea  in df_hist_ca:
    hist_text_feature[fea+"_hist_text"]=[]
hist_text_feature['card_id']=[]

for cid,cid_df in  gp:
    hist_text_feature['card_id'].append(cid)
    for  fea in  df_hist_ca:
        if fea=='card_id':
            continue
        hist_text_feature[fea+"_hist_text"].append(" ".join(cid_df[fea].astype(str).tolist()))

df_hist_text=pd.DataFrame(hist_text_feature)
df_hist_text

,authorized_flag_hist_text,category_1_hist_text,installments_hist_text,category_3_hist_text,merchant_category_id_hist_text,merchant_id_hist_text,category_2_hist_text,state_id_hist_text,subsector_id_hist_text,purchase_amount_hist_text,card_id
0,1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...,3 3 1 3 6 1 3 3 1 1 1 3 1 8 3 3 1 4 2 10 2 3 3...,C C B C C B C C B B B C B C C C B C C C C C C ...,422 884 884 884 68 884 68 506 884 884 884 884 ...,M_ID_112e15a77b M_ID_d13262f4b0 M_ID_7c26306ee...,1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1....,9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 -1 9 9 9...,27 27 27 27 27 27 27 30 27 27 27 27 38 7 27 27...,-0.40159855 -0.22699049 -0.62684597 -0.3685402...,C_ID_0894217f2f
1,0 1 0 1 1 1 1 0 0 1 1 1 0 0 1 1 0 0 0 1 1 1 1 ...,0 0 1 0 0 0 0 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 ...,1 1 4 1 1 1 1 6 1 1 2 2 4 1 1 1 5 1 1 1 1 1 1 ...,B B C B B B B C B B C C C B B B C B B B B B B ...,412 278 879 80 278 307 879 879 80 298 879 511 ...,M_ID_85a5803161 M_ID_c873d839bd M_ID_9139332cc...,1.0 1.0 nan 1.0 1.0 1.0 1.0 nan 1.0 1.0 nan na...,12 12 -1 12 12 12 9 -1 12 12 -1 -1 -1 12 12 12...,18 37 29 37 37 19 29 29 37 10 29 7 29 37 19 19...,-0.69732031 -0.71685477 -0.48008203 -0.7439024...,C_ID_0ab67a22ab
2,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 ...,1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,B B B B B C B B B B B B B B B B B B B B B B B ...,705 705 705 705 130 87 367 705 195 130 705 307...,M_ID_4d13bdcf15 M_ID_c3ba8a0906 M_ID_7e6e285f1...,4.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 4....,13 13 13 13 13 13 13 13 13 13 13 13 13 13 17 1...,33 33 33 33 41 27 16 33 34 41 33 19 33 33 19 1...,0.3188175 -0.58716095 -0.69299267 -0.63425404 ...,C_ID_130fd0cbdd
3,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 ...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 ...,B B B B B B B B B B B B B B B B B B C B B B B ...,195 683 80 651 454 278 879 386 278 80 879 879 ...,M_ID_309752ddea M_ID_5533f08da2 M_ID_39c4515cb...,4.0 4.0 4.0 1.0 4.0 4.0 nan 4.0 4.0 4.0 1.0 1....,22 22 22 16 22 22 -1 22 22 22 9 9 22 9 22 9 16...,34 34 37 2 39 37 29 29 37 37 29 29 37 29 27 29...,-0.71685477 -0.72082177 -0.68633593 -0.6597540...,C_ID_186d6a6901
4,1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,B B B B B B B B B B B B B B B B B B B B B B B ...,360 437 307 222 222 437 332 307 568 796 544 68...,M_ID_9e9a33e52b M_ID_fd02f8d662 M_ID_90fff6242...,4.0 3.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 3.0 4.0 4....,13 1 13 13 13 13 13 13 13 1 13 13 13 13 13 13 ...,34 15 19 21 21 15 2 19 29 10 29 34 27 19 15 33...,-0.66876993 -0.51504872 -0.70182826 -0.0556882...,C_ID_2b5e3df5c2
5,1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 ...,3 1 5 1 1 1 1 1 1 1 1 1 3 4 5 1 1 1 1 1 1 3 6 ...,C B C B B B B B B B B B C C C B B B B B B C C ...,68 528 68 80 278 278 879 560 307 560 80 560 68...,M_ID_879f625c89 M_ID_124bc5f024 M_ID_ac1b9b1c8...,1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1....,9 9 9 9 15 9 9 9 9 9 9 9 9 9 9 9 9 -1 9 -1 -1 ...,27 25 27 37 37 37 29 34 19 34 37 34 27 17 27 3...,-0.59679294 -0.694315 -0.34870523 -0.66275933 ...,C_ID_3d0044924f
6,0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0,0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0,1 1 4 1 1 1 1 1 1 1 1 2 1 2 1 1,B B C B B B B B B B B C B C B B,307 307 879 307 307 307 307 879 879 307 606 84...,M_ID_1c5c0b7fde M_ID_c5574af2f7 M_ID_00a6ca8a8...,1.0 3.0 1.0 nan 3.0 3.0 3.0 1.0 nan 1.0 1.0 1....,16 19 9 -1 19 19 19 9 -1 16 9 16 16 16 16 16,19 19 29 19 19 19 19 29 29 19 17 20 19 20 19 19,0.1396564 -0.50798626 0.30494803 0.1396564 -0....,C_ID_5814b4f13c
7,0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 ...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,B B B B B B B B B B B B B B B B B B B B B B B ...,307 45 45

In [153]:
new_mer_text_feature={}
gp=df_new_merchant_trans.groupby('card_id')
for fea  in df_hist_ca:
    new_mer_text_feature[fea+"_new_mer_text"]=[]
new_mer_text_feature['card_id']=[]
for cid,cid_df in  gp:
    new_mer_text_feature['card_id'].append(cid)
    for  fea in  df_hist_ca:
        if fea=='card_id':
            continue
        new_mer_text_feature[fea+"_new_mer_text"].append(" ".join(cid_df[fea].astype(str).tolist()))

In [154]:
df_new_mer_text=pd.DataFrame(new_mer_text_feature)
df_new_mer_text

,authorized_flag_new_mer_text,category_1_new_mer_text,installments_new_mer_text,category_3_new_mer_text,merchant_category_id_new_mer_text,merchant_id_new_mer_text,category_2_new_mer_text,state_id_new_mer_text,subsector_id_new_mer_text,purchase_amount_new_mer_text,card_id
0,1 1 1 1,0 0 0 0,5 2 3 1,C C C B,427 383 506 69,M_ID_a5cec05aae M_ID_4077dbee56 M_ID_f4be79819...,1.0 1.0 1.0 1.0,9 9 9 9,27 2 30 20,-9.016e-05 -0.47174231 -0.27373797 -0.65449475,C_ID_0894217f2f
1,1 1 1,0 0 0,3 1 1,C B B,411 307 80,M_ID_4942754bdb M_ID_963c3bc06b M_ID_be0a17ad20,1.0 1.0 1.0,12 12 12,21 19 37,-0.38326621 -0.67177524 -0.72211405,C_ID_0ab67a22ab
2,1 1 1 1 1 1 1 1 1,0 0 1 0 0 0 0 0 1,3 1 1 1 1 1 1 1 1,C B B B B B B B B,650 531 879 367 443 278 367 422 574,M_ID_2e5f4fb265 M_ID_fa067e91a0 M_ID_00a6ca8a8...,4.0 4.0 nan 4.0 4.0 4.0 4.0 4.0 nan,13 13 -1 13 13 13 13 13 -1,29 27 29 16 21 37 16 27 22,-0.50648361 -0.65674872 -0.61181945 -0.6483188...,C_ID_130fd0cbdd
3,1 1 1 1 1 1 1,0 0 0 0 1 0 0,-1 1 1 1 1 1 1,nan B B B B B B,416 278 114 80 839 80 68,M_ID_db86b8ff60 M_ID_aa13f8b4d9 M_ID_7177ec37b...,4.0 4.0 4.0 4.0 nan 4.0 4.0,22 22 22 22 -1 22 22,8 37 7 37 29 37 27,-0.56673992 -0.70047587 -0.71685477 -0.7341352...,C_ID_186d6a6901
4,1 1 1 1 1 1,0 0 0 0 0 0,1 -1 1 1 2 1,B nan B B C B,278 373 360 667 108 278,M_ID_255bc75465 M_ID_964968046f M_ID_c6aa40fef...,4.0 3.0 4.0 4.0 4.0 4.0,13 1 13 13 13 13,37 18 34 21 34 37,-0.70408224 0.49277943 -0.67177524 -0.63721426...,C_ID_2b5e3df5c2
5,1 1 1 1 1 1,0 0 0 0 0 0,1 1 1 1 1 1,B B B B B B,514 278 307 307 705 80,M_ID_6e7c412a33 M_ID_90c3d91284 M_ID_4dbadbd1c...,1.0 1.0 1.0 1.0 1.0 1.0,9 9 9 9 9 9,9 37 19 19 33 37,-0.73188128 -0.71536715 -0.73338393 -0.7394095...,C_ID_3d0044924f
6,1,0,1,B,307,M_ID_a529bfcd5a,3.0,2,19,-0.17590035,C_ID_5814b4f13c
7,1 1 1 1 1 1 1 1 1,0 0 0 0 0 0 0 0 0,1 1 1 1 1 1 1 1 1,B B B B B B B B B,842 667 108 705 506 45 307 307 705,M_ID_8dd3f2e24a M_ID_b87364a9ad M_ID_1db8cc906...,1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0,9 9 9 9 9 9 9 9 9,37 21 34 33 30 18 19 19 33,-0.67478054 -0.67628319 -0.65867212 -0.6587172...,C_ID_6d8dba8475
8,1 1 1 1 1,0 0 0 0 0,0 0 0 0 0,A A A A A,561 19 369 63 703,M_ID_f2045dd267 M_ID_95eff8501c M_ID_c92209cec...,5.0 5.0 5.0 5.0 5.0,21 21 21 21 21,7 36 27 27 29,-0.72484888 -0.74202418 -0.72451829 -0.6883494...,C_ID_7e63323c00
9,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0,A A A A A A A A A A A A A A A A A A A A A A A,80 80 278 842 783 454 278 683 278 307 130 278 ...,M_ID_98b342c0e3 M_ID_28a21ea733 M_ID_20dce166b...,1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1....,9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9,37 37 37 37 19 39 37 34 37 19 41 37 37 29 18 3...,-0.49145709 -0.66656104 -0.63524579 -0.5811804...,C_ID_92a2005557


In [158]:
df_train=pd.merge(df_train,df_hist_text,how='left',on='card_id')
df_test=pd.merge(df_test,df_hist_text,how='left',on='card_id')

df_all=pd.concat([df_train,df_test])

In [165]:
print(df_all[df_all['card_id']=='C_ID_0894217f2f']['authorized_flag_hist_text'])

5    1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
Name: authorized_flag_hist_text, dtype: object
